<a href="https://colab.research.google.com/github/priyal6/NLP-Prac/blob/main/retrieve%26rerank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers rank_bm25

In [2]:
import gzip
import json
import os

import torch

from sentence_transformers import CrossEncoder, SentenceTransformer, util

if not torch.cuda.is_available():
  print("Warning: No GPU found. Please add GPU to your notebook")

bi_encoder = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")
bi_encoder.max_seq_length = 256
top_k = 32 #no of item we retrieve

cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L6-v2")

wikipedia_filepath = "simplewiki-2020-11-01.jsonl.gz"

if not os.path.exists(wikipedia_filepath):
  util.http_get("http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz", wikipedia_filepath)

passages= []
with gzip.open(wikipedia_filepath, "rt", encoding = "utf8") as fIn:
  for line in fIn:
    data = json.loads(line.strip())

    passages.append(data["paragraphs"][0])

print("Passages:", len(passages))

corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

  0%|          | 0.00/50.2M [00:00<?, ?B/s]

Passages: 169597


Batches:   0%|          | 0/5300 [00:00<?, ?it/s]

In [4]:
def search(query):
  print("Input question:", query)


  question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
  question_embedding = question_embedding.cuda()
  hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
  hits = hits[0]

  cross_inp = [[query, passages[hit["corpus_id"]]] for hit in hits]
  cross_scores = cross_encoder.predict(cross_inp)

  for idx in range(len(cross_scores)):
    hits[idx]["cross-score"] = cross_scores[idx]


  print("Top-3 Bi-Encoder Retrieval hits")
  hits = sorted(hits, key= lambda x: x["score"], reverse=True)
  for hit in hits[0:3]:
    print("\t{:.3f}\t{}".format(hit["score"], passages[hit["corpus_id"]].replace("\n", " ")))


  print("Top-3 Cross-Encoder Re-Ranker hits")
  hits = sorted(hits, key= lambda x: x["cross-score"], reverse=True)
  for hit in hits[0:3]:
     print("\t{:.3f}\t{}".format(hit["cross-score"], passages[hit["corpus_id"]].replace("\n", " ")))


In [5]:
search(query="What is the capital of the United States?")

Input question: What is the capital of the United States?
Top-3 Bi-Encoder Retrieval hits
	0.622	Cities in the United States:
	0.597	The United States Capitol is the building where the United States Congress meets. It is the center of the legislative branch of the U.S. federal government. It is in Washington, D.C., on top of Capitol Hill at the east end of the National Mall.
	0.596	In the United States:
Top-3 Cross-Encoder Re-Ranker hits
	8.906	Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. The President of the USA and many major national government offices are in the territory. This makes it the political center of the United States of America.
	3.755	A capital city (or capital town or just capital) is a city or town, specified by law or constitution, by the government of a country, or part of a country, such as a state, province or county. It usually serves as the lo

In [6]:
search(query="How many people live in Toronto?")

Input question: How many people live in Toronto?
Top-3 Bi-Encoder Retrieval hits
	0.604	Vaughan is a city in Ontario, Canada, 335,000 people live there .
	0.595	Toronto is a city in Ohio in the United States.
	0.594	Toronto is a city in Kansas, United States.
Top-3 Cross-Encoder Re-Ranker hits
	4.448	It has about 110,000 people living there.
	3.632	The 2010 census counted 23,302 people living in the city. (2005 showed 24,709). The city was founded on January 1, 1955. Much of the city was heavily damaged by the 2011 Tōhoku earthquake and tsunami.
	3.372	As of January 1, 2010, about 164,294 people lived there. That was 295.83 people per km². The total area is 555.35 km².


In [7]:
search(query="What is reinforcement learning?")

Input question: What is reinforcement learning?
Top-3 Bi-Encoder Retrieval hits
	0.749	Reinforcement learning (RL) is teaching a "software agent" how to behave in an environment by telling it how good it's doing. It is an area of machine learning inspired by behaviorist psychology.
	0.490	One way to learn new skills is by observing the behavior of other people or animals. This is called Observational learning. It is a form of social learning which can take different forms. In humans, this form of learning seems to not need reinforcement; instead a social model such as a parent, sibling, friend, or teacher with surroundings is needed. For a child, a model is someone of authority or higher status in an environment. In animals, observational learning is often based on classical conditioning. In classical conditioning instinctive behavior is provoked by observing the behavior of another (e.g. mobbing in birds), but other processes may be involved as well.
	0.451	A reward is getting somethi

In [9]:
search(query = "What is sequence level tokenisation in comparasion to RL?")

Input question: What is sequence level tokenisation in comparasion to RL?
Top-3 Bi-Encoder Retrieval hits
	0.420	Reinforcement learning (RL) is teaching a "software agent" how to behave in an environment by telling it how good it's doing. It is an area of machine learning inspired by behaviorist psychology.
	0.393	PSL can mean several things:
	0.384	HLS has a number of meanings:
Top-3 Cross-Encoder Re-Ranker hits
	-5.620	In mathematics, a sequence is an ordered set of mathematical objects (most notably a list of numbers). In some cases, the sequence tends towards a limit, in which case the limit is denoted using the symbol formula_1 (as in formula_2), and the sequence is said to be convergent, otherwise it is divergent.
	-6.456	Reinforcement learning (RL) is teaching a "software agent" how to behave in an environment by telling it how good it's doing. It is an area of machine learning inspired by behaviorist psychology.
	-8.367	An analog signal processor is used to quickly process sign